In [ ]:
# @title ▶️ Run this cell to start the Downloader App
import os
import shutil
import sys

# 1. Install dependencies silently
print("Installing dependencies... (this may take a minute)")
os.system('pip install yt-dlp gradio')
print("Dependencies installed!")

import gradio as gr
import yt_dlp

def sanitize_filename(name):
    """Removes bad characters from filenames."""
    return "".join([c for c in name if c.isalnum() or c in (' ', '-', '_')]).strip()

def downloader_process(url_text, filename_text, format_type, quality_setting):
    # Setup directories
    base_folder = "/content/downloads"
    zip_name = "/content/audio_bundle"

    # Clean up previous runs
    if os.path.exists(base_folder):
        shutil.rmtree(base_folder)
    os.makedirs(base_folder)

    if os.path.exists(f"{zip_name}.zip"):
        os.remove(f"{zip_name}.zip")

    # Parse Inputs
    urls = [line.strip() for line in url_text.split('\n') if line.strip()]
    names = [line.strip() for line in filename_text.split('\n') if line.strip()]

    logs = []

    # Validation
    if len(urls) == 0:
        return "❌ Error: No URLs provided.", None
    if len(urls) != len(names):
        return f"❌ Error: Count mismatch!\nURLs: {len(urls)}\nFilenames: {len(names)}\nPlease ensure both lists have the same number of lines.", None

    logs.append(f"Starting download of {len(urls)} files...")

    # Download Loop
    for url, raw_name in zip(urls, names):
        clean_name = sanitize_filename(raw_name)
        if not clean_name: clean_name = "audio_track"

        logs.append(f"⏳ Processing: {clean_name}...")

        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': f'{base_folder}/{clean_name}.%(ext)s',
            'ignoreerrors': True,
            'no_warnings': True,
            'quiet': True,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': format_type,
                'preferredquality': quality_setting,
            }],
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            logs.append(f"✅ Success: {clean_name}")
        except Exception as e:
            logs.append(f"❌ Failed: {clean_name} ({str(e)})")

    # Zip the result
    logs.append("\n📦 Zipping files...")
    shutil.make_archive(zip_name, 'zip', base_folder)
    logs.append("✨ Done! Download your zip file below.")

    return "\n".join(logs), f"{zip_name}.zip"

# UI Layout
with gr.Blocks(title="YouTube Audio Downloader") as demo:
    gr.Markdown("# 🎵 Batch Audio Downloader")
    gr.Markdown("Paste your links and filenames below (one per line).")

    with gr.Row():
        with gr.Column():
            input_urls = gr.Textbox(label="List of URLs (One per line)", lines=5, placeholder="https://youtube.com/...\nhttps://youtube.com/...")
            input_names = gr.Textbox(label="List of Filenames (One per line)", lines=5, placeholder="My Song 1\nMy Song 2")

            with gr.Row():
                fmt_dropdown = gr.Dropdown(choices=["mp3", "m4a", "wav", "flac"], value="mp3", label="Format")
                qual_dropdown = gr.Dropdown(choices=["128", "192", "320"], value="192", label="Quality (kbps)")

            btn = gr.Button("🚀 Start Download", variant="primary")

        with gr.Column():
            status_output = gr.Textbox(label="Status Log", lines=10)
            file_output = gr.File(label="Download ZIP")

    btn.click(
        fn=downloader_process,
        inputs=[input_urls, input_names, fmt_dropdown, qual_dropdown],
        outputs=[status_output, file_output]
    )

# Launch
demo.launch(debug=True)

Installing dependencies... (this may take a minute)
Dependencies installed!
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2a2151c16b0c1cb37f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
